In [2]:
%load_ext Cython

In [4]:
%%cython
def _dist_2(double x0, double y0, double x1, double y1):

    return c_dist_2(x0, y0, x1, y1)

cdef double c_dist_2(double x0, double y0, double x1, double y1):

    cdef double d0
    cdef double d1

    d0 = x1 - x0
    d1 = y1 - y0

    return d0 * d0 + d1 * d1

In [5]:
%timeit _dist_2(1000, 1000, 5000, 5000)

The slowest run took 6.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 477 ns per loop


In [ ]:
from scipy.spatial.distance import euclidean

%timeit euclidean([1000, 1000], [5000, 5000])

The slowest run took 33.02 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 36.7 µs per loop


In [9]:
%%cython
from libc.math cimport sqrt

def _circumcircle_radius(pt1, pt2, pt3):
    
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3
    
    return c_circumcircle_radius(x0, y0, x1, y1, x2, y2)

cdef double c_dist_2(double x0, double y0, double x1, double y1):

    cdef:
        double d0 = x1 - x0
        double d1 = y1 - y0

    return d0 * d0 + d1 * d1

cdef double _distance(double x0, double y0, double x1, double y1):
    
    return sqrt(c_dist_2(x0, y0, x1, y1))

cdef double c_circumcircle_radius(double x0, double y0, double x1, double y1, double x2, double y2):

    cdef:
        double a = _distance(x0, y0, x1, y1)
        double b = _distance(x1, y1, x2, y2)
        double c = _distance(x2, y2, x0, y0)
        double s = (a + b + c) * 0.5
        double prod = s*(a+b-s)*(a+c-s)*(b+c-s)
        double prod2 = a*b*c
        double radius = prod2 * prod2 / (16*prod)

    return radius

In [10]:
%timeit _circumcircle_radius([0, 0], [10.1, 10.1], [10, 0])

The slowest run took 5.63 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 750 ns per loop


In [7]:
import math

def dist_2(x0, y0, x1, y1):

    d0 = x1 - x0
    d1 = y1 - y0
    return d0 * d0 + d1 * d1

def distance(p0, p1):

    return math.sqrt(dist_2(p0[0], p0[1], p1[0], p1[1]))

def circumcircle_radius(pt0, pt1, pt2):

    a = distance(pt0, pt1)
    b = distance(pt1, pt2)
    c = distance(pt2, pt0)

    s = (a + b + c) * 0.5

    prod = s * (s - a) * (s - b) * (s - c)
    prod2 = a * b * c

    radius = prod2 / (4 * math.sqrt(prod))

    return radius

In [8]:
%timeit circumcircle_radius([0, 0], [10.1, 10.1], [10, 0])

100000 loops, best of 3: 5.21 µs per loop


In [24]:
%%cython

def _circumcenter(pt1, pt2, pt3):
        
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3
        
    return c_circumcenter(x0, y0, x1, y1, x2, y2)

cdef c_circumcenter(double x0, double y0, double x1, double y1, double x2, double y2):

    cdef:
        double bc_y_diff = y1 - y2
        double ca_y_diff = y2 - y0
        double ab_y_diff = y0 - y1
        double d_inv = 0.5 / (x0 * bc_y_diff + x1 * ca_y_diff + x2 * ab_y_diff)
        double a_mag = x0 * x0 + y0 * y0
        double b_mag = x1 * x1 + y1 * y1
        double c_mag = x2 * x2 + y2 * y2
        double cx
        double cy

    cx = (a_mag * bc_y_diff + b_mag * ca_y_diff + c_mag * ab_y_diff) * d_inv
    cy = (a_mag * (x2 - x1) + b_mag * (x0 - x2) + c_mag * (x1 - x0)) * d_inv
    
    return cx, cy


In [25]:
%timeit _circumcenter([0, 0], [10.1, 10.1], [10, 0])

The slowest run took 5.40 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 782 ns per loop


In [27]:
def circumcenter(pt0, pt1, pt2):

    a_x = pt0[0]
    a_y = pt0[1]
    b_x = pt1[0]
    b_y = pt1[1]
    c_x = pt2[0]
    c_y = pt2[1]

    bc_y_diff = b_y - c_y
    ca_y_diff = c_y - a_y
    ab_y_diff = a_y - b_y
    cb_x_diff = c_x - b_x
    ac_x_diff = a_x - c_x
    ba_x_diff = b_x - a_x

    d_div = (a_x * bc_y_diff + b_x * ca_y_diff + c_x * ab_y_diff)

    if d_div == 0:
        raise ZeroDivisionError

    d_inv = 0.5 / (a_x * bc_y_diff + b_x * ca_y_diff + c_x * ab_y_diff)

    a_mag = a_x * a_x + a_y * a_y
    b_mag = b_x * b_x + b_y * b_y
    c_mag = c_x * c_x + c_y * c_y

    cx = (a_mag * bc_y_diff + b_mag * ca_y_diff + c_mag * ab_y_diff) * d_inv
    cy = (a_mag * cb_x_diff + b_mag * ac_x_diff + c_mag * ba_x_diff) * d_inv

    return cx, cy

In [28]:
%timeit circumcenter([0, 0], [10.1, 10.1], [10, 0])

100000 loops, best of 3: 4.21 µs per loop
